# Test Network Hyperparameters

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
import itertools
from sklearn import metrics


In [5]:
df = pd.read_csv('finalDF.csv',index_col=0)
df = df.drop(columns=['glucose'])
display(df.head(5))

,hadm_id,subject_id,expire_flag,admission_type,elix_score,age,admission_type_encoded,glucose_readings,0,1,...,21,22,23,24,25,26,27,28,29,range
0,137275,9319,0,ELECTIVE,12,82,0,45,118.0,97.0,...,133.0,142.0,142.0,99.0,99.0,98.0,126.0,109.0,109.0,82.0
1,125380,31585,0,EMERGENCY,7,300,1,10,118.0,127.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
2,132722,1426,0,EMERGENCY,5,80,1,9,104.0,104.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0
3,101422,71582,0,EMERGENCY,0,76,1,21,106.0,134.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.0
4,139687,68501,0,ELECTIVE,10,86,0,5,162.0,180.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0


In [6]:
from sklearn.utils import resample
df_majority = df[df.expire_flag==0]
df_minority = df[df.expire_flag==1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,   
                                 n_samples=3000,     
                                 random_state=123) 

df_majority_downsampled = resample(df_majority, 
                                 replace=False, 
                                 n_samples=5000,     
                                 random_state=123) 

df_resampled = pd.concat([df_minority_upsampled, df_majority_downsampled])

In [7]:
pydf = df_resampled.values
print(pydf.shape)
X = pydf[:,4:39]
y = df_resampled['expire_flag']
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X = X.reshape((X.shape[0], X.shape[1], 1))

(8000, 39)


c:\users\ian\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [137]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier

def mortality_model():
    model = Sequential()
    model.add(Conv1D(filters=100, kernel_size=2, activation='relu', input_shape=(35, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=mortality_model,epochs=50,batch_size=32)   

In [138]:
model.fit(X_train,y_train,epochs=50, batch_size=32)

Epoch 1/50
5600/5600 [==============================] - 5s 979us/step - loss: 0.2224 - acc: 0.6357
Epoch 2/50
5600/5600 [==============================] - 2s 287us/step - loss: 0.2038 - acc: 0.6743
Epoch 3/50
5600/5600 [==============================] - 2s 282us/step - loss: 0.1953 - acc: 0.6871
Epoch 4/50
5600/5600 [==============================] - 2s 283us/step - loss: 0.1894 - acc: 0.7016
Epoch 5/50
5600/5600 [==============================] - 2s 284us/step - loss: 0.1862 - acc: 0.7087
Epoch 6/50
5600/5600 [==============================] - 2s 299us/step - loss: 0.1833 - acc: 0.7189
Epoch 7/50
5600/5600 [==============================] - 2s 296us/step - loss: 0.1802 - acc: 0.7202
Epoch 8/50
5600/5600 [==============================] - 2s 287us/step - loss: 0.1780 - acc: 0.7234
Epoch 9/50
5600/5600 [==============================] - 2s 295us/step - loss: 0.1761 - acc: 0.7311
Epoch 10/50
5600/5600 [==============================] - 2s 286us/step - loss: 0.1765 - acc: 0.7268
Epoch 11/

## Batch Size and Epochs

In [139]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=mortality_model)   
batch_size = [10,20,50]
epochs = [5]
parameters = {'batch_size': batch_size,'epochs': epochs}
clf = GridSearchCV(model,parameters)
clf.fit(X_train,y_train)

Epoch 1/5
3733/3733 [==============================] - 7s 2ms/step - loss: 0.2184 - acc: 0.6421
Epoch 2/5
3733/3733 [==============================] - 3s 727us/step - loss: 0.2056 - acc: 0.6713
Epoch 3/5
3733/3733 [==============================] - 3s 734us/step - loss: 0.1980 - acc: 0.6887
Epoch 4/5
3733/3733 [==============================] - 3s 748us/step - loss: 0.1928 - acc: 0.6981
Epoch 5/5
3733/3733 [==============================] - 1s 269us/step
Epoch 1/5
3733/3733 [==============================] - 7s 2ms/step - loss: 0.2175 - acc: 0.6499
Epoch 2/5
3733/3733 [==============================] - 3s 759us/step - loss: 0.2028 - acc: 0.6716
Epoch 3/5
3733/3733 [==============================] - 3s 764us/step - loss: 0.1970 - acc: 0.6898
Epoch 4/5
3733/3733 [==============================] - 3s 755us/step - loss: 0.1934 - acc: 0.6941
Epoch 5/5
3733/3733 [==============================] - 1s 283us/step
Epoch 1/5
3734/3734 [==============================] - 7s 2ms/step - loss: 0.2175 

GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001ECA774E4A8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'batch_size': [10, 20, 50], 'epochs': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [140]:
print(clf.best_score_,clf.best_params_)

0.7046428591598357 {'batch_size': 10, 'epochs': 5}


## Optimizer

In [141]:
def mortality_model(optimizer):
    model = Sequential()
    model.add(Conv1D(filters=100, kernel_size=2, activation='relu', input_shape=(35, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse',metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=mortality_model,epochs=20,batch_size=10)  
parameters = {'optimizer':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']}
clf = GridSearchCV(model,parameters)
clf.fit(X_train,y_train)

Epoch 1/20
3733/3733 [==============================] - 7s 2ms/step - loss: 0.2339 - acc: 0.6263
Epoch 2/20
3733/3733 [==============================] - 3s 725us/step - loss: 0.2238 - acc: 0.6322
Epoch 3/20
3733/3733 [==============================] - 3s 779us/step - loss: 0.2200 - acc: 0.6432
Epoch 4/20
3733/3733 [==============================] - 3s 789us/step - loss: 0.2163 - acc: 0.6464
Epoch 5/20
3733/3733 [==============================] - 3s 787us/step - loss: 0.2133 - acc: 0.6453
Epoch 6/20
3733/3733 [==============================] - 3s 768us/step - loss: 0.2111 - acc: 0.6510
Epoch 7/20
3733/3733 [==============================] - 3s 679us/step - loss: 0.2091 - acc: 0.6606
Epoch 8/20
3733/3733 [==============================] - 3s 696us/step - loss: 0.2081 - acc: 0.6571
Epoch 9/20
3733/3733 [==============================] - 3s 716us/step - loss: 0.2064 - acc: 0.6692
Epoch 10/20
3733/3733 [==============================] - 3s 702us/step - loss: 0.2061 - acc: 0.6635
Epoch 11/20

3733/3733 [==============================] - 3s 789us/step - loss: 0.1780 - acc: 0.7235
Epoch 17/20
3733/3733 [==============================] - 3s 791us/step - loss: 0.1774 - acc: 0.7254
Epoch 18/20
3733/3733 [==============================] - 3s 788us/step - loss: 0.1769 - acc: 0.7265
Epoch 19/20
3733/3733 [==============================] - 3s 787us/step - loss: 0.1764 - acc: 0.7292
Epoch 20/20
3733/3733 [==============================] - 1s 305us/step
Epoch 1/20
3734/3734 [==============================] - 7s 2ms/step - loss: 0.2093 - acc: 0.6749
Epoch 2/20
3734/3734 [==============================] - 3s 791us/step - loss: 0.1967 - acc: 0.6915
Epoch 3/20
3734/3734 [==============================] - 3s 790us/step - loss: 0.1920 - acc: 0.6958
Epoch 4/20
3734/3734 [==============================] - 3s 789us/step - loss: 0.1871 - acc: 0.7081
Epoch 5/20
3734/3734 [==============================] - 3s 791us/step - loss: 0.1850 - acc: 0.7217
Epoch 6/20
3734/3734 [==========================

3733/3733 [==============================] - 5s 1ms/step - loss: 0.1837 - acc: 0.7134
Epoch 11/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1822 - acc: 0.7252
Epoch 12/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1806 - acc: 0.7163
Epoch 13/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1794 - acc: 0.7308A: 1
Epoch 14/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1777 - acc: 0.7351
Epoch 15/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1765 - acc: 0.7289
Epoch 16/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1744 - acc: 0.7380
Epoch 17/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1726 - acc: 0.7394
Epoch 18/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1708 - acc: 0.7407
Epoch 19/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1701 - acc: 0.7442
Epoch 20/20
3733/3733 [=====

GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001ECD7FACD68>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [142]:
print(clf.best_score_, clf.best_params_)

0.725357142750706 {'optimizer': 'Adam'}


## Activation Functions

In [144]:
def mortality_model(activation):
    model = Sequential()
    model.add(Conv1D(filters=100, kernel_size=2, activation=activation, input_shape=(35, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=mortality_model,epochs=20,batch_size=10)  
parameters = {'activation':['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']}
clf = GridSearchCV(model,parameters)
clf.fit(X_train,y_train)

Epoch 1/20
3733/3733 [==============================] - 10s 3ms/step - loss: 0.2480 - acc: 0.6255
Epoch 2/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2346 - acc: 0.6255
Epoch 3/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2345 - acc: 0.6255
Epoch 4/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2345 - acc: 0.6255
Epoch 5/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2345 - acc: 0.6255
Epoch 6/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2343 - acc: 0.6255
Epoch 7/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2340 - acc: 0.6255
Epoch 8/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2339 - acc: 0.6255
Epoch 9/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2332 - acc: 0.6255
Epoch 10/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2319 - acc: 0.6255
Epoch 11/20
3733/3733 [=====

3733/3733 [==============================] - 4s 1ms/step - loss: 0.1909 - acc: 0.6970
Epoch 20/20
3733/3733 [==============================] - 2s 562us/step
Epoch 1/20
3734/3734 [==============================] - 11s 3ms/step - loss: 0.2194 - acc: 0.6580
Epoch 2/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.2036 - acc: 0.6797
Epoch 3/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.2038 - acc: 0.6784
Epoch 4/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.2021 - acc: 0.6834
Epoch 5/20
3734/3734 [==============================] - 6s 1ms/step - loss: 0.2006 - acc: 0.6875
Epoch 6/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.2000 - acc: 0.6888
Epoch 7/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1967 - acc: 0.6939
Epoch 8/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1978 - acc: 0.6901
Epoch 9/20
3734/3734 [==============================] - 6s 1ms/ste

3733/3733 [==============================] - 6s 2ms/step - loss: 0.2363 - acc: 0.6255
Epoch 17/20
3733/3733 [==============================] - 6s 1ms/step - loss: 0.2357 - acc: 0.6255
Epoch 18/20
3733/3733 [==============================] - 6s 2ms/step - loss: 0.2363 - acc: 0.6255
Epoch 19/20
3733/3733 [==============================] - 6s 2ms/step - loss: 0.2372 - acc: 0.6255
Epoch 20/20
3733/3733 [==============================] - 3s 671us/step
Epoch 1/20
3733/3733 [==============================] - 11s 3ms/step - loss: 0.2519 - acc: 0.6022
Epoch 2/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2362 - acc: 0.6258
Epoch 3/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2347 - acc: 0.6258
Epoch 4/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2364 - acc: 0.6258
Epoch 5/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2363 - acc: 0.6258
Epoch 6/20
3733/3733 [==============================] - 5s 1ms/

3734/3734 [==============================] - 5s 1ms/step - loss: 0.1954 - acc: 0.6939
Epoch 14/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1932 - acc: 0.6993
Epoch 15/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1937 - acc: 0.6987
Epoch 16/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1936 - acc: 0.6958
Epoch 17/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1934 - acc: 0.6987
Epoch 18/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1958 - acc: 0.6918
Epoch 19/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1948 - acc: 0.6987A: 0s - loss: 0.192
Epoch 20/20
3734/3734 [==============================] - 3s 726us/step
Epoch 1/20
5600/5600 [==============================] - 14s 2ms/step - loss: 0.2115 - acc: 0.6563
Epoch 2/20
5600/5600 [==============================] - 7s 1ms/step - loss: 0.1964 - acc: 0.6895
Epoch 3/20
5600/5600 [===================

GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001ECD7FACCF8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'activation': ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [145]:
print(clf.best_score_, clf.best_params_)

0.7189285726765436 {'activation': 'relu'}


## Kernal Size

In [146]:
def mortality_model(kernel_size):
    model = Sequential()
    model.add(Conv1D(filters=100, kernel_size=kernel_size, activation='relu', input_shape=(35, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=mortality_model,epochs=20,batch_size=10)  
parameters = {'kernel_size':[2,3,4,6,8]}
clf = GridSearchCV(model,parameters)
clf.fit(X_train,y_train)

Epoch 1/20
3733/3733 [==============================] - 11s 3ms/step - loss: 0.2198 - acc: 0.6424
Epoch 2/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.2056 - acc: 0.6678
Epoch 3/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1988 - acc: 0.6823
Epoch 4/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1949 - acc: 0.6927
Epoch 5/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1934 - acc: 0.6992
Epoch 6/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1897 - acc: 0.7056
Epoch 7/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1858 - acc: 0.7088
Epoch 8/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1830 - acc: 0.7187
Epoch 9/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1812 - acc: 0.7246
Epoch 10/20
3733/3733 [==============================] - 5s 1ms/step - loss: 0.1778 - acc: 0.7329
Epoch 11/20
3733/3733 [=====

3733/3733 [==============================] - 2s 531us/step
Epoch 1/20
3734/3734 [==============================] - 11s 3ms/step - loss: 0.2174 - acc: 0.6559
Epoch 2/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1991 - acc: 0.6864
Epoch 3/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1922 - acc: 0.6974
Epoch 4/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1875 - acc: 0.7145
Epoch 5/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1836 - acc: 0.7196
Epoch 6/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1819 - acc: 0.7196
Epoch 7/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1786 - acc: 0.7258
Epoch 8/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1762 - acc: 0.7220
Epoch 9/20
3734/3734 [==============================] - 5s 1ms/step - loss: 0.1763 - acc: 0.7314
Epoch 10/20
3734/3734 [==============================] - 5s 1ms/ste

5600/5600 [==============================] - 8s 1ms/step - loss: 0.1543 - acc: 0.7700
Epoch 18/20
5600/5600 [==============================] - 8s 1ms/step - loss: 0.1519 - acc: 0.7743
Epoch 19/20
5600/5600 [==============================] - 8s 1ms/step - loss: 0.1485 - acc: 0.7823
Epoch 20/20
5600/5600 [==============================] - 8s 1ms/step - loss: 0.1459 - acc: 0.7845


GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001ECD7FACB00>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel_size': [2, 3, 4, 6, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [147]:
print(clf.best_score_, clf.best_params_)

0.7339285710241114 {'kernel_size': 8}
